In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2


In [2]:
path = r'C:\Users\fajar\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK049024,Karma Resorts,Other,Past Guests,Karma Kandara,PF260716,Ms,Nicole,Satherley,Female,...,2024-08-11 00:00:00,2,HOUSE_USE,V_OWNER,COMP,6,1,0,Email and Phone,NO
1,KK052446,Karma Resorts,Other,Past Guests,Karma Kandara,PF266920,Mr,Lucien,Corbeau,Male,...,2024-08-13 00:00:00,5,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
2,KK045285,Karma Resorts,Other,Past Guests,Karma Kandara,PF253729,Mr,Frances,Gurnett,Female,...,2024-08-11 00:00:00,2,OTA,OTA,RETAIL,4,0,1,Email and Phone,NO
3,KK044997,Karma Resorts,Other,Past Guests,Karma Kandara,PF253055,Ms,Rose,Mannion,Female,...,2024-08-16 00:00:00,4,OTA,OTA,RETAIL,7,0,0,Phone Only,NO
4,KK058272,Karma Resorts,Other,Past Guests,Karma Kandara,PF280220,Mr,Dean,Mobley,Male,...,2024-08-19 00:00:00,4,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,KK054540,Karma Resorts,Other,Past Guests,Karma Kandara,PF270867,Mrs,Elias,Copin,Female,...,2024-08-13 00:00:00,2,OTA,OTA,RETAIL,6,2,0,Phone Only,NO
59,KSH013567,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF229227,<NA>,Gina,Tye,Unknown,...,2024-08-18 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
60,KLM006200,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF281993,Ms,Bella,Campbell,Unknown,...,2024-08-11 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
61,KSH013109,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF244371,Mrs,Amanda,Walters,Female,...,2024-08-19 00:00:00,2,DIR,WEDDING,GROUP,2,0,0,Email Only,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

In [7]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
12,KB041447,Karma Resorts,Other,Past Guests,Karma Bavaria,PF268412,Frau,Patricia,Weissbart Ott,Female,...,2024-08-11 00:00:00,1,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
13,KB041445,Karma Resorts,Other,Past Guests,Karma Bavaria,PF268412,Frau,Patricia,Weissbart Ott,Female,...,2024-08-11 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
19,KB041875,Karma Resorts,Other,Past Guests,Karma Bavaria,PF213986,Ms,Sabine,Reiner,Female,...,2024-08-17 00:00:00,1,COMP,V_OWNER,COMP,2,0,0,Email and Phone,NO
28,KB041906,Karma Resorts,Other,Past Guests,Karma Bavaria,PF271001,Herr,Stefan,Krafft,Male,...,2024-08-17 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
29,KB041906,Karma Resorts,Other,Past Guests,Karma Bavaria,PF271001,Herr,Stefan,Krafft,Male,...,2024-08-17 00:00:00,1,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
40,KK051385,Karma Resorts,Other,Past Guests,Karma Kandara,PF265296,Mr,Lucien Jean Claude,Corbeau,Male,...,2024-08-14 00:00:00,7,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
49,KSM011588,Karma Resorts,Other,Past Guests,Karma St Martins,PF258217,Mrs,Gemma,Sherlock,Unknown,...,2024-08-11 00:00:00,2,COMP,STAFF,COMP,2,0,0,Email and Phone,NO
54,KSM011421,Karma Resorts,Other,Past Guests,Karma St Martins,PF147242,empty,Nish,Goodhand,Unknown,...,2024-08-17 00:00:00,5,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
56,KSM009393,Karma Resorts,Other,Past Guests,Karma St Martins,PF098426,Mr,Steve,Backshall,Unknown,...,2024-08-13 00:00:00,2,COMP,STAFF,COMP,1,0,0,Email and Phone,NO
58,KK054540,Karma Resorts,Other,Past Guests,Karma Kandara,PF270867,Mrs,Elias,Copin,Female,...,2024-08-13 00:00:00,2,OTA,OTA,RETAIL,6,2,0,Phone Only,NO


In [8]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK049024,Karma Resorts,Other,Past Guests,Karma Kandara,PF260716,Ms,Nicole,Satherley,Female,...,2024-08-11 00:00:00,2,HOUSE_USE,V_OWNER,COMP,6,1,0,Email and Phone,NO
1,KK052446,Karma Resorts,Other,Past Guests,Karma Kandara,PF266920,Mr,Lucien,Corbeau,Male,...,2024-08-13 00:00:00,5,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
2,KK045285,Karma Resorts,Other,Past Guests,Karma Kandara,PF253729,Mr,Frances,Gurnett,Female,...,2024-08-11 00:00:00,2,OTA,OTA,RETAIL,4,0,1,Email and Phone,NO
3,KK044997,Karma Resorts,Other,Past Guests,Karma Kandara,PF253055,Ms,Rose,Mannion,Female,...,2024-08-16 00:00:00,4,OTA,OTA,RETAIL,7,0,0,Phone Only,NO
4,KK058272,Karma Resorts,Other,Past Guests,Karma Kandara,PF280220,Mr,Dean,Mobley,Male,...,2024-08-19 00:00:00,4,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
5,KK058588,Karma Resorts,Other,Past Guests,Karma Kandara,PF280946,Mr,Lekhyananda,Piwat,Male,...,2024-08-14 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
6,KK054679,Karma Resorts,Other,Past Guests,Karma Kandara,PF271164,Mrs,Alana,Cuthbert,Female,...,2024-08-12 00:00:00,2,DIR,CHAT,DIGITAL,6,0,0,Email Only,NO
7,KB043833,Karma Resorts,Other,Past Guests,Karma Bavaria,PF273258,Mrs,Charlotte,Vesselle,Unknown,...,2024-08-13 00:00:00,1,COMP,TA,RETAIL,2,0,0,Phone Only,NO
8,KB042371,Karma Resorts,Other,Past Guests,Karma Bavaria,PF273799,Dr,Verena,Mattheis,Female,...,2024-08-19 00:00:00,2,DIR,EMAIL_MRKTNG,DIGITAL,3,0,0,Email and Phone,NO
9,KB043595,Karma Resorts,Other,Past Guests,Karma Bavaria,PF280998,Mrs,Josephine,Evangelista,Female,...,2024-08-11 00:00:00,2,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO


In [9]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [10]:
find_member_booking.to_csv(r'C:\Users\fajar\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [11]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
16,KB040103,Karma Resorts,Other,Past Guests,Karma Bavaria,PF213986,Ms,Sabine,Reiner,Female,...,2024-08-18 00:00:00,4,COMP,GER_OWE,COMP,3,0,0,Email and Phone,NO


In [12]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [13]:
count_data_akhir = df_analytic.shape[0]

In [14]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK049024,Karma Resorts,Other,Past Guests,Karma Kandara,PF260716,Ms,Nicole,Satherley,Female,...,2024-08-11 00:00:00,2,HOUSE_USE,V_OWNER,COMP,6,1,0,Email and Phone,NO
1,KLM006200,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF281993,Ms,Bella,Campbell,Unknown,...,2024-08-11 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
2,KSM013518,Karma Resorts,Other,Past Guests,Karma St Martins,PF281133,Mrs,Helen,Smith,Unknown,...,2024-08-11 00:00:00,3,DIR,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
3,KSM013248,Karma Resorts,Other,Past Guests,Karma St Martins,PF277844,Mr,George,Mccahill,Unknown,...,2024-08-11 00:00:00,1,COMP,WALKIN,RETAIL,2,0,0,Email and Phone,NO
4,KSM011587,Karma Resorts,Other,Past Guests,Karma St Martins,PF258217,Mrs,Gemma,Sherlock,Unknown,...,2024-08-11 00:00:00,2,COMP,STAFF,COMP,2,0,0,Email and Phone,NO
5,KSH019370,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF235873,Mrs,Rebekah,Bristow,Female,...,2024-08-11 00:00:00,1,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
6,KK056239,Karma Resorts,Other,Past Guests,Karma Kandara,PF104987,Mr,Shahid,Salimi,Male,...,2024-08-11 00:00:00,1,DIR,CHAT,DIGITAL,6,0,0,Email and Phone,NO
7,KSM012328,Karma Resorts,Other,Past Guests,Karma St Martins,PF268671,Mrs,Isabella,Dascola,Unknown,...,2024-08-11 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
8,KK055913,Karma Resorts,Other,Past Guests,Karma Kandara,PF281184,Mr,Matteo,Angeretti,Male,...,2024-08-11 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
9,KB037013,Karma Resorts,Other,Past Guests,Karma Bavaria,PF248950,Frau,Karin,Knoll,Female,...,2024-08-11 00:00:00,3,OTA,OTA,RETAIL,2,1,1,Email and Phone,NO


In [15]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [16]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


In [17]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK049024,Karma Resorts,Other,Past Guests,Karma Kandara,PF260716,Ms,Nicole,Satherley,Female,...,2024-08-11,2,HOUSE_USE,V_OWNER,COMP,6,1,0,Email and Phone,NO
1,KB036990,Karma Resorts,Other,Past Guests,Karma Bavaria,PF248952,Herr,Alexander,Franke,Male,...,2024-08-11,3,OTA,OTA,RETAIL,2,1,1,Email and Phone,NO
2,KB043107,Karma Resorts,Other,Past Guests,Karma Bavaria,PF278203,Frau,Tina,Hees,Female,...,2024-08-11,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
3,KB041446,Karma Resorts,Other,Past Guests,Karma Bavaria,PF268412,Frau,Patricia,Weissbart Ott,Female,...,2024-08-11,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
4,KB040084,Karma Resorts,Other,Past Guests,Karma Bavaria,PF262250,Mrs,Josephin,Gleixner,Female,...,2024-08-11,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
5,KK045285,Karma Resorts,Other,Past Guests,Karma Kandara,PF253729,Mr,Frances,Gurnett,Female,...,2024-08-11,2,OTA,OTA,RETAIL,4,0,1,Email and Phone,NO
6,KB043595,Karma Resorts,Other,Past Guests,Karma Bavaria,PF280998,Mrs,Josephine,Evangelista,Female,...,2024-08-11,2,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
7,KB037016,Karma Resorts,Other,Past Guests,Karma Bavaria,PF249039,Frau,Birgit,Franke,Female,...,2024-08-11,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
8,KB043124,Karma Resorts,Other,Past Guests,Karma Bavaria,PF278376,Herr,Elmar,Schobel,Male,...,2024-08-11,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
9,KB038973,Karma Resorts,Other,Past Guests,Karma Bavaria,PF257133,,Stephanie,Fischl,Unknown,...,2024-08-11,1,OTA,OTA,RETAIL,4,0,0,Phone Only,NO


In [18]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 63
Jumlah Data Yang Duplicate : 10
Jumlah Data Setelah Hapus Duplicate : 53
Jumlah Data Yang Termasuk Member: 1
Jumlah Data Setelah Hapus Member : 52
Jumlah Data Akhir : 52


In [19]:
df_analytic.to_excel(r'C:\Users\fajar\Documents\Python\Data\final_analytics.xlsx', index=False)